In [2]:
import transformers
import os
import re
from collections import Counter
from transformers import BertConfig, BertForMaskedLM, BertTokenizer, DataCollatorForLanguageModeling, Trainer, TrainingArguments
from datasets import Dataset
# create a matrix
from scipy.sparse import lil_matrix
import numpy as np
from foobar import *


In [12]:
del corpus_file

In [3]:
# Load your corpus
with open('he_wiki_cleaned.txt', 'r', encoding='utf-8') as f:
    corpus_file = [line.strip() for line in f if line.strip()]

In [16]:
# Step 1: Create Vocabulary with Frequency Threshold

def create_vocabulary(corpus_file):
    vocab = Counter()
    for line in corpus_file:
        vocab.update(Counter(line.strip().split()))
    
    # Filter tokens by frequency threshold
    vocab_pre = dict([(token, count) for token, count in vocab.items()])
    
    # Create a vocabulary dictionary with tokens mapped to (index, frequency)
    return {token: (i, count) for i, (token, count) in enumerate(vocab_pre.items())}


In [17]:
# Create the vocabulary
vocab = create_vocabulary(corpus_file)
np.save("vocab", vocab)

In [18]:
vocab

{'political': (0, 842317),
 'philosophy': (1, 151189),
 'and': (2, 80574397),
 'movement': (3, 443699),
 'anarchism': (4, 5528),
 'is': (5, 25122875),
 'a': (6, 57551679),
 'that': (7, 15952892),
 'against': (8, 1989324),
 'all': (9, 3252345),
 'forms': (10, 265911),
 'of': (11, 93262519),
 'authority': (12, 247376),
 'seeks': (13, 34054),
 'to': (14, 57024021),
 'abolish': (15, 8439),
 'the': (16, 197926562),
 'institutions': (17, 164408),
 'it': (18, 12786844),
 'claims': (19, 181253),
 'maintain': (20, 111970),
 'unnecessary': (21, 17284),
 'coercion': (22, 4901),
 'hierarchy': (23, 24363),
 'typically': (24, 227303),
 'including': (25, 2066258),
 'state': (26, 2363638),
 'capitalism': (27, 18041),
 'advocates': (28, 30268),
 'for': (29, 23322864),
 'replacement': (30, 103570),
 'with': (31, 20624213),
 'stateless': (32, 3012),
 'societies': (33, 57976),
 'voluntary': (34, 35430),
 'free': (35, 524123),
 'associations': (36, 75780),
 'as': (37, 23057737),
 'historically': (38, 88552

In [23]:
def check_word_pairs_in_corpus(word_pairs, corpus_vocab):
    count = 0
    included_pairs = []
    
    for pair, score in word_pairs:
        word1, word2 = pair
        # Check if both words in the pair are present in the corpus
        if word1 in corpus_vocab and word2 in corpus_vocab:
            count += 1
            included_pairs.append(pair)
    
    return count, included_pairs

In [25]:
# 2002 word pairs for Rare dataset
test_files = ['wordsim353.txt', 'men_dataset.txt', 'mturk.txt', 'rarewords.txt', 'simlex999.txt']

for i in range(len(test_files)):
    path_input = "D:\\20240901paper3\\similarities\\"+test_files[i]
    test_word_pairs_set = read_test_word_pairs_set(path_input)
    included = check_word_pairs_in_corpus(test_word_pairs_set, vocab)
    print(included[0])

353
3000
287
2002
999


In [19]:
# There are in total 144581193 sentences
len(corpus_file)

144581193

In [ ]:
np.sum(corpus_file)

In [21]:
# There are in total 10582162 unique words
len(vocab)

10582162

In [22]:
corpus_file[0:10]

['political philosophy and movement anarchism is a political philosophy and movement that is against all forms of authority and seeks to abolish the institutions it claims maintain unnecessary coercion and hierarchy typically including the state and capitalism',
 'anarchism advocates for the replacement of the state with stateless societies and voluntary free associations',
 'as a historically leftwing movement this reading of anarchism is placed on the farthest left of the political spectrum usually described as the libertarian wing of the socialist movement libertarian socialism',
 'although traces of anarchist ideas are found all throughout history modern anarchism emerged from the enlightenment',
 'during the latter half of the th and the first decades of the th century the anarchist movement flourished in most parts of the world and had a significant role in workers struggles for emancipation',
 'various anarchist schools of thought formed during this period',
 'anarchists have ta

In [28]:
# When threshold is 100, there are in total 293391 unique words
vocabthr100 = [word for word, freq in vocab.items() if freq[1] >= 100]
len(vocabthr100)

293391

In [ ]:
import random

In [36]:
# When threshold is 100, there are in total 1573 word pairs for Rare dataset
vocab_temp = vocabthr100
vocab_pro = [1, 0.2, 0.1, 0.05]
test_files = ['wordsim353.txt', 'men_dataset.txt', 'mturk.txt', 'rarewords.txt', 'simlex999.txt']
for j in range(len(vocab_pro)):
    print(vocab_pro[j])
    random.seed(4)
    print(np.round(len(vocab_temp)*vocab_pro[j]).astype(int))

    vocab_temp_sample = random.sample(vocab_temp, np.round(len(vocab_temp)*vocab_pro[j]).astype(int))
    for i in range(len(test_files)):
        path_input = "D:\\20240901paper3\\similarities\\"+test_files[i]
        test_word_pairs_set = read_test_word_pairs_set(path_input)
        not_included = check_word_pairs_in_corpus(test_word_pairs_set, vocab_temp_sample)
        print(not_included[0])

1
293391
353
3000
285
1573
998
0.2
58678
13
120
15
64
31
0.1
29339
8
33
4
18
8
0.05
14670
0
8
2
5
5


In [37]:
len(vocab_temp)*vocab_pro[3]

14669.550000000001

In [38]:
# When threshold is 1000, there are in total 64856 unique words

vocabthr1000 = [word for word, freq in vocab.items() if freq[1] >= 1000]
len(vocabthr1000)

64856

In [39]:
# When threshold is 1000, there are in total 785 word pairs for Rare dataset

vocab_temp = vocabthr1000
vocab_pro = [1, 0.2, 0.1, 0.05]
test_files = ['wordsim353.txt', 'men_dataset.txt', 'mturk.txt', 'rarewords.txt', 'simlex999.txt']
for j in range(len(vocab_pro)):
    print(vocab_pro[j])
    random.seed(4)
    print(np.round(len(vocab_temp)*vocab_pro[j]).astype(int))

    vocab_temp_sample = random.sample(vocab_temp, np.round(len(vocab_temp)*vocab_pro[j]).astype(int))
    for i in range(len(test_files)):
        path_input = "D:\\20240901paper3\\similarities\\"+test_files[i]
        test_word_pairs_set = read_test_word_pairs_set(path_input)
        not_included = check_word_pairs_in_corpus(test_word_pairs_set, vocab_temp_sample)
        print(not_included[0])

1
64856
353
2982
284
785
992
0.2
12971
15
170
11
37
49
0.1
6486
2
51
2
6
10
0.05
3243
0
14
1
1
2


In [40]:
# Save the vocabulary to a file
vocab_file_path = 'vocabthr1000.txt'
with open(vocab_file_path, 'w', encoding='utf-8') as vocab_file:
    for token in vocabthr1000:
        vocab_file.write(f"{token}\n")

In [3]:
vocab_file_path = 'vocabthr1000.txt'

with open(vocab_file_path, 'r', encoding='utf-8') as vocab_file:
    vocabthr1000 = [line.strip() for line in vocab_file]

In [5]:
len(vocab)

10582162

In [7]:
len(vocabthr1000)

64856

In [9]:
# Filter the dictionary based on words in vocabthr1000.txt
vocabthr1000_dic = {word: value for word, value in vocab.items() if word in vocabthr1000}
np.save("vocabthr1000_dic", vocabthr1000_dic)

In [10]:
# index starts from 0
unique_values = [value[0] for value in vocabthr1000_dic.values()]
print(unique_values)
value_to_index = {val: idx for idx, val in enumerate(unique_values)}
vocabthr1000_dic_index = {key: (value_to_index[value[0]], value[1]) for key, value in vocabthr1000_dic.items()}
np.save("vocabthr1000_dic_index", vocabthr1000_dic_index)
list(vocabthr1000_dic_index.items())[-5:]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 173, 174, 175, 176, 177, 178, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 215, 216, 217, 218, 219, 220, 221, 222, 223, 224, 225, 226, 227,

[('won–lostole', (64851, 1350)),
 ('won–lostsouth', (64852, 1248)),
 ('won–losttennesee', (64853, 1350)),
 ('won–losttexas', (64854, 1348)),
 ('won–lostvanderbilt', (64855, 1326))]